In [8]:
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
labels = pd.read_csv("Xray_TeethLabels_Simple.csv",index_col=0)
Ylabels = labels[5:]
metaY = labels[:3]
X = pd.DataFrame()
Y = []
for picFilename in glob.glob("Processing3/processed/*"):
    #print(picFilename)
    patNumber = picFilename.split('/')[2].split('_')[0]
    #toothNumber = picFilename.split('/')[2].split('_')[1].split('.')[0]
    toothNumber1 = picFilename.split('/')[2].split('_')[1]
    toothNumber2 = picFilename.split('/')[2].split('_')[2].split('.')[0]
    im = Image.open(picFilename)
    #X[patNumber + '_' + toothNumber] = np.array(im).flatten()
    #Y.append(Ylabels.loc[toothNumber,patNumber])
    
    X[patNumber + '_' + toothNumber1 + '_' + toothNumber2] = np.array(im).flatten()
    if Ylabels.loc[toothNumber1,patNumber] == 'Yes' or Ylabels.loc[toothNumber2,patNumber] == 'Yes':
        Y.append(1)
    else:
        Y.append(0)

X_train, X_test, y_train, y_test = train_test_split(X.T.to_numpy(), np.array(Y), test_size=0.20)

In [17]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))
print(y_train)

135
34
135
34
[0 1 1 1 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1 0 1 0
 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 0 1 1]


In [18]:
np.random.seed(6)
def svc_param_selection(X, y, nfolds):
    C = [0.1, 1, 10, 100, 1000] #try different values for C
    param_grid = {'C': C}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_
best_c = svc_param_selection(X.T.to_numpy(), np.array(Y), 2)
print ("The best value for C is:",best_c['C'])

The best value for C is: {'C': 10}


In [14]:
model = SVC(C=best_c['C'], kernel='rbf')
model.fit(X_train, y_train)
print(model)
# make predictions
expected = y_test
predicted = model.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

SVC(C=10)
              precision    recall  f1-score   support

           0       0.38      0.21      0.27        14
           1       0.58      0.75      0.65        20

    accuracy                           0.53        34
   macro avg       0.48      0.48      0.46        34
weighted avg       0.49      0.53      0.50        34

[[ 3 11]
 [ 5 15]]


In [15]:
np.random.seed(6)
def svc_param_selection(X, y, nfolds):
    C = [0.1, 1, 10, 100, 1000] #try different values for C
    param_grid = {'C': C}
    grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_
best_c = svc_param_selection(X.T.to_numpy(), np.array(Y), 2)
print ("The best value for C is:",best_c['C'])

The best value for C is: {'C': 0.1}


In [16]:
model = SVC(C=best_c['C'], kernel='linear')
model.fit(X_train, y_train)
print(model)
# make predictions
expected = y_test
predicted = model.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

SVC(C=0.1, kernel='linear')
              precision    recall  f1-score   support

           0       0.50      0.21      0.30        14
           1       0.61      0.85      0.71        20

    accuracy                           0.59        34
   macro avg       0.55      0.53      0.50        34
weighted avg       0.56      0.59      0.54        34

[[ 3 11]
 [ 3 17]]
